In [49]:
import gym
import numpy as np
import time
import random
import operator
from IPython.display import clear_output
from sklearn.decomposition import PCA
from gym import wrappers
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import multiprocessing
from deap import gp

In [14]:
env = gym.make("AirRaid-ram-v0")
#env = wrappers.Monitor(env, '/tmp/cartpole-experiment-5')
env.reset();

In [15]:
env.step(1)

(array([188, 184, 180, 184, 188,  62, 188,   0,   0,   0, 128,  16,  56,
        212, 146,  56,  68,   0,  37,  64,  39, 119,   0, 159, 252, 159,
        252,   0,   0, 158, 158, 158, 135, 109, 135,   0,   0,   0, 224,
          2,   0,   0,   0,  15,   5,  15, 240, 246, 240, 246, 240, 246,
        240, 246, 248, 246, 248, 246,   0,   0,   2,   2,   2,   3,   6,
          2,  28,  75, 127,   2,   2,   2,   0,   0,   0,   0,   0,   0,
        214, 187, 214,   0,   0,   0,   0,  40,  72,   2,   0,   3,   0,
          0,   6, 252,   4,   4,  64,   0,   0,   0,   0,   0,   0,   0,
        255, 255,   0,   0,   0,   0,   0,   0, 135, 247, 214, 247, 109,
        247, 187, 247, 158, 158,   0,   0, 202, 245, 144, 245], dtype=uint8),
 0.0,
 False,
 {'ale.lives': 1})

In [16]:
# for i in range(100):
#     env.step(random.randint(0,1))
#     env.render()
# env.reset()

In [40]:
pset = gp.PrimitiveSet("main", 128)
def if_then_else(input, output1, output2):
    return output1 if input else output2

pset.addPrimitive(if_then_else, 3)

pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(max, 2)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.mul, 2)

In [41]:
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)

In [42]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [43]:
def sample_reward(env, func, t_max=500):
    s = env.reset()
    total_reward = 0
    for _ in range(t_max):

        next_action = func(*s)
        next_action = np.clip(next_action,0,5)
        next_action = int(abs(next_action))
        s, reward, is_done, _ = env.step(next_action)
        total_reward += reward
        if is_done:
            break
    return total_reward

In [44]:
def evalSymbReg(individual, n_times=100):
    
    func = toolbox.compile(expr=individual)
    rewards = [sample_reward(env, func) for _ in range(n_times)]
    
    return float(np.mean(rewards)),

In [50]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=16)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=17)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

In [51]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

In [ ]:
pop = toolbox.population(n=5)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)

In [144]:
best_func = gp.compile(hof[0], pset)

In [141]:
evalSymbReg(hof[0])

(500.0,)

In [ ]:
for i in range(100):
    s = env.reset()
    total_reward = 0
    for _ in range(500):
        next_action = best_func(*s)
        next_action = 0 if next_action<=0 else 1
        s, reward, is_done, _ = env.step(next_action)
        total_reward += reward
        env.render()
        if is_done:
            break


RecursionError: maximum recursion depth exceeded while calling a Python object